# Students Do: Predicting Fraudulent Loans Applications

According to the American Bankers Association, [_"every dollar of fraud now costs banks and credit unions roughly $2.92"_](https://www.aba.com/member-tools/industry-solutions/insights/state-card-fraud-2018), that's a reason why predicting fraud using machine learning techniques becomes a [broad area of research](https://scholar.google.com.mx/scholar?q=fraud+detection+machine+learning&btnG=&oq=fraud+detection+) and a great [business opportunity for FinTech startups](https://www.eu-startups.com/2019/06/paris-based-fintech-bleckwen-raises-e8-8-million-for-its-fraud-detection-software-to-prevent-financial-crime/).

In this activity, you are going to explore how tree based algorithms can be used to identify fraudulent loan applications. You will start using a decision tree model, that will be trained with the `sba_loans_encoded.csv` file that you created before.

In [3]:
# Initial imports
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Needed for decision tree visualization
#import pydotplus
from IPython.display import Image


## Loading and Preprocessing Loans Encoded Data

Load the `sba_loans_encoded.csv` in a pandas DataFrame called `df_loans`.

In [5]:
path = Path("../Resources/sba_loans_encoded.csv")

df_loans = pd.read_csv(path)

df_loans


,Year,Month,Amount,Term,Zip,CreateJob,NoEmp,RealEstate,RevLineCr,UrbanRural,...,City_WILLITS,City_WILMINGTON,City_WINDSOR,City_WINNETKA,City_WOODLAND,City_WOODLAND HILLS,City_WRIGHTWOOD,City_Watsonville,City_YORBA LINDA,City_YUBA CITY
0,2001,11,32812,36,92801,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2001,4,30000,56,90505,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2001,4,30000,36,92103,0,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2003,10,50000,36,92108,0,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,7,343000,240,91345,3,65,1,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,2006,6,150000,60,92346,0,5,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2093,1997,4,99000,300,92021,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2094,1997,2,50000,84,93012,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2095,1997,1,251150,120,91352,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Define the features set, by copying the `df_loans` DataFrame and dropping the `Default` column.

In [7]:
x = df_loans.copy()

x.drop('Default', axis=1, inplace=True)
x


,Year,Month,Amount,Term,Zip,CreateJob,NoEmp,RealEstate,RevLineCr,UrbanRural,...,City_WILLITS,City_WILMINGTON,City_WINDSOR,City_WINNETKA,City_WOODLAND,City_WOODLAND HILLS,City_WRIGHTWOOD,City_Watsonville,City_YORBA LINDA,City_YUBA CITY
0,2001,11,32812,36,92801,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2001,4,30000,56,90505,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2001,4,30000,36,92103,0,10,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2003,10,50000,36,92108,0,6,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2006,7,343000,240,91345,3,65,1,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2092,2006,6,150000,60,92346,0,5,0,0,2,...,0,0,0,0,0,0,0,0,0,0
2093,1997,4,99000,300,92021,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2094,1997,2,50000,84,93012,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2095,1997,1,251150,120,91352,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create the target vector by assigning the values of the `Default` column from the `df_loans` DataFrame.

In [10]:
# Define target vector
y = df_loans['Default'].values.reshape(-1,1)

Split the data into training and testing sets.

In [11]:
# Splitting into Train and Test sets
x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=78)


Use the `StandardScaler` to scale the features data, remember that only `X_train` and `X_testing` DataFrames should be scaled.

In [12]:
# Create the StandardScaler instance
scaler = StandardScaler()


In [14]:
# Fit the Standard Scaler with the training data
x_scaler = scaler.fit(x_train)


In [15]:
# Scale the training data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

## Fitting the Decision Tree Model

Once data is scaled, create a decision tree instance and train it with the training data (`X_train_scaled` and `y_train`).

In [16]:
# Create the decision tree classifier instance
model = tree.DecisionTreeClassifier()


In [17]:
# Fit the model

model = model.fit(x_train_scaled, y_train)

## Making Predictions Using the Tree Model

Validate the trained model, by predicting fraudulent loan applications using the testing data (`X_test_scaled`).

In [18]:
# Making predictions using the testing data

prediction = model.predict(x_test_scaled)

## Model Evaluation

Evaluate model's results, by using `sklearn` to calculate the confusion matrix, the accuracy score and to generate the classification report.

In [20]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, prediction)
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])


# Calculating the accuracy score
acc_score = accuracy_score(y_test, prediction)


In [22]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, prediction))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,325,19
Actual 1,26,155


Accuracy Score : 0.9142857142857143
Classification Report
              precision    recall  f1-score   support

           0       0.93      0.94      0.94       344
           1       0.89      0.86      0.87       181

    accuracy                           0.91       525
   macro avg       0.91      0.90      0.90       525
weighted avg       0.91      0.91      0.91       525



## Visualizing the Decision Tree

In this section, you should create a visual representation of the decision tree using `pydotplus`. Show the graph on the notebook, and also save it in `PDF` and `PNG` formats.

In [24]:
# Create DOT data
dot_data = tree.export_graphviz(
    model, out_file=None, feature_names=x.columns, class_names=["0", "1"], filled=True
)

# Draw graph
graph = pydotplus.graph_from_dot_data(dot_data)

# Show graph
Image(graph.create_png())



NameError: name 'pydotplus' is not defined

In [ ]:
# Saving the tree as PDF


# Saving the tree as PNG



## Analysis Question

Finally, analyze the model's evaluation results and answer the following question.

* Would you trust in this model to deploy a loans application approval solution in a bank?

 * **Your answer here**